In [138]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash import Dash, dcc, html, Input, Output,callback

In [139]:
data = pd.read_json("dataset.json")
data_to_see = data.iloc[10]
data_to_see

stations          {'Златоуст (1)': ['0', '2', '1', '29', '13', '...
full_timetable    {'667': {'route': ['1', '3', '2'], 'free_carri...
Name: 10, dtype: object

In [324]:
hat = [item[:-4] for item in data_to_see['stations'].keys()]
data_to_see[1]
hat

['Златоуст',
 'Кыштым',
 'Миасс',
 'Муслюмово',
 'Челябинск',
 'Полетаево',
 'Еманжелинск']

In [ ]:
1 - номер поезда
2 - время прибытия
3 - Время отбытия
4 - количество вагонов для сцепки
5 - сколько вагонов везет 

In [325]:
from datetime import datetime, time
def_data = list(interval.values())[:5]
start_time = datetime.strptime('01:10', '%H:%M').time()
def_data2 = [(datetime.strptime(t, '%H:%M') - datetime.combine(datetime.today(), start_time)) for t in def_data]
def_data2_str = [str(((dt.days * 24 + dt.seconds) // 3600) % 24 - 1).zfill(2) + ':' + str((dt.seconds // 60) % 60).zfill(2) for dt in def_data2] 
train  = ['№ '+item for item in data_to_see['full_timetable'].keys()]
dict_train = {'Поезд':train[:5], 'Время прибытия': def_data, 'Время отбытия':def_data2_str,
        'Количество вагонов для сцепки':[2,0,7,9,0],
       'Сколько вагонов везет':[12,34,28,24,4], 'Следующая станция':hat[:5]}
column_for_table = dict.keys()
data_far_table = dict.values()
df_for_table = pd.DataFrame.from_dict(dict_train) 
df_for_table

,Поезд,Время прибытия,Время отбытия,Количество вагонов для сцепки,Сколько вагонов везет,Следующая станция
0,№ 667,02:10,10:00,2,12,Златоуст
1,№ 247,07:24,16:14,0,34,Кыштым
2,№ 903,03:38,12:28,7,28,Миасс
3,№ 111,01:22,10:12,9,24,Муслюмово
4,№ 451,02:10,10:00,0,4,Челябинск


In [334]:
app = dash.Dash(__name__)
app.layout = html.Div(
    style={'background-image': 'url("https://catherineasquithgallery.com/uploads/posts/2021-02/thumbs/1613682224_1-p-fon-dlya-prezentatsii-zheleznaya-doroga-1.jpg")',
           'background-size': 'cover',
           'width': '100%',
           'height': '110vh'},
    children=[
        html.Br(),
        html.H1(children='Координация пропуска вагонопотока',
                style={'fontSize':'36px', 'textAlign':'center'}),
        html.Div([
            html.Label('Выберете экземпляр вагонопотока'),
            dcc.Input(
                id='input1',
                type='number',
                min=1,
                max=100000,
                step='1'
            ),
            html.Br(),
            html.Label('Выберете время стоянки поезда для каждой станции'),
            dcc.Input(
                id='input2',
                type='text',
                pattern=r'^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$'
            ),
            html.Br(),
            html.Label('Выберите станцию'),
            dcc.Dropdown(hat,
                id='drop1',
                multi=False
            ),
            html.Div(id='output1'),
            html.Br(),
            html.Button('Посмотреть вагонопоток', id='button', n_clicks=0,
                        style={'fontSize':'14px', 'width': '150px',
                               'fontWeight': 'bold', 'height': '100'}),
            html.Br(),
            html.Div(
                children=[
                    html.H1(children='Выполненные расчеты',
                            style={'fontSize':'24px', 'textAlign':'center'}),
                    dash_table.DataTable(
                            data=df_for_table.to_dict('records'),
                            columns=[{"name": i, "id": i} for i in df_for_table.columns],
                        style_cell={'textAlign': 'left', 'minWidth': '150px',
                                    'whiteSpace': 'normal', 'height': 'auto',
                                    'fontSize':'12px'},
                                    style_header={'textAlign': 'center', 'fontWeight': 'bold', 'fontSize':'14px'}
                        )

                ],
                style={'width': '100%', 'margin': '0 auto'}
            )
        ]
        )
    ]
)

@app.callback(
    Output('output1', 'children'),
    Input('drop1', 'value')
)
def update_drop(station):
    if station:
        return f'Вы сотрудник станции {station}'
    else:
        return 'Выберите станцию для расписания вагонопотока'

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True)

In [263]:
from datetime import datetime
timetable_values = [item['timetable'] for item in data_to_see['full_timetable'].values()]
time_range=[]
for i in range(len(timetable_values)):
    lis=[]
    for j in range(len(timetable_values[i])-1):
        x = datetime.strptime(timetable_values[i][j][-5::], '%H:%M').time()
        y = datetime.strptime(timetable_values[i][j+1][:5], '%H:%M').time()
        a = datetime.combine(datetime.min, y) - datetime.combine(datetime.min, x)
        lis.append(a)
    time_range.append(lis)
routes = [item['route'] for item in data_to_see['full_timetable'].values()]
routes_range = []
for i in range(len(routes)):
    r=[]
    for j in range(len(routes[i])-1):
        lt = f"{routes[i][j]}-{routes[i][j+1]}"
        r.append(lt)
    routes_range.append(r)
combine_dict = {}
unique_routes = set(sum(routes_range, []))
unique_timetable = set(sum(time_range, []))
combined_list=[]
for i in range(len(routes_range)):
    combine_dict.update({s:x for s,x in zip(routes_range[i], time_range[i])})
interval = {}
for key, value in combine_dict.items():
    total_seconds = value.total_seconds()
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    interval[key] = f'{int(hours):02d}:{int(minutes):02d}'
start_time=[]
for i in range(len(timetable_values)):
    a = timetable_values[i][0][:5]
    start_time.append(a)
start_value = {}
for i, j in zip(train, start_time):
    start_value[i] = j
interval

{'1-3': '02:10',
 '3-2': '07:24',
 '3-6': '03:38',
 '6-5': '01:22',
 '5-4': '02:10',
 '5-7': '02:38',
 '2-3': '07:24',
 '2-4': '08:00',
 '4-5': '02:10',
 '4-2': '08:00',
 '3-1': '02:10',
 '5-6': '01:22',
 '6-3': '03:38',
 '7-5': '02:38',
 '7-6': '01:06'}

In [277]:
mass = np.zeros(shape=(len(start_value),len(interval)+2), dtype='object')
mass[:,0] = list(start_value.keys())
mass[:,1] = list(start_value.values())
for i in range(len(mass)):
    for j in range(len(interval)):
                   mass[j,2+i] = list(interval.values())

IndexError: index 17 is out of bounds for axis 1 with size 17